In [1]:
sc

In [2]:
sc

In [3]:
lines = sc.textFile("input/file1")
lines

input/file1 MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:-2

In [4]:
lines.collect()

[u'Hello World Bye World']

In [5]:
words = lines.flatMap(lambda l: l.split(" "))
words.count()

4

# Problem 1

In [29]:
path = "/home/vagrant/csds-material/input"
lines = sc.textFile(path)
record = lines.flatMap(lambda l: l.split(" "))\
            .map(lambda a: (a, 1))\
            .reduceByKey(lambda x,y:x+y)\
            .map(lambda x:(x[0],x[1]))\
            .sortByKey(True)
Ans = record.take(record.count())
for a in Ans:
    print a[0] + ' : ' + str(a[1])

Bye : 1
Goodbye : 1
Hadoop : 2
Hello : 2
World : 2


# Problem 2

In [57]:
from pyspark.sql.types import *

sqlContext = SQLContext(sc)

# 1. Load the purchases.txt file into an RDD
path = "/home/vagrant/csds-material/hive/purchases.txt"
lines = sc.textFile(path)
record = lines.map(lambda l: l.split(","))

# 2. create a DataFrame from the RDD
purchases = record.map(lambda p: (p[0].split(" ")[0], p[1], p[2], float(p[3]), p[4]))

fields = [StructField("timestamp", StringType(), True), \
          StructField("location", StringType(), True), \
          StructField("category", StringType(), True), \
          StructField("price", FloatType(), True), \
          StructField("card", StringType(), True), \
         ]

purchasesSchema = StructType(fields)
purchasesDataFrame = sqlContext.createDataFrame(purchases, purchasesSchema)
purchasesTable = purchasesDataFrame.registerTempTable("purchases")

# 3. formuate the queries for answering the questions

# i. What is the average price of the products that were purchased via Mastercard?
query = "select avg(price) as Ans from purchases where card = \"MasterCard\" "
result = sqlContext.sql(query)
Ans = result.map(lambda a: str(a.Ans))
print "i. What is the average price of the products that were purchased via Mastercard?"
print float(Ans.collect()[0])
print ''

# ii. Which date recorded the highest total sales?
query = "select timestamp as DATE, sum(price) as SALE from purchases group by timestamp order by SALE DESC LIMIT 1"
result = sqlContext.sql(query)
Ans = result.map(lambda a: str(a.DATE))
print "ii. Which date recorded the highest total sales?"
print Ans.collect()[0]
print ''

# iii. What is the minimum value of a product under the Computers category?
query = "select min(price) as Ans from purchases where category = \"Computers\" "
result = sqlContext.sql(query)
Ans = result.map(lambda a: a.Ans)
print "iii. What is the minimum value of a product under the Computers category?"
print round(float(Ans.collect()[0]),2)
print ''

# iv. How many distinct categories of products are there?
query = "select count(*) as Ans from (select DISTINCT category from purchases) temp "
result = sqlContext.sql(query)
Ans = result.map(lambda a: str(a.Ans))
print "iv. How many distinct categories of products are there?"
print int(Ans.collect()[0])
print ''

# v. Which store location had the lowest total sales?
query = "select location as Ans, sum(price) as SALE from purchases group by location order by SALE ASC LIMIT 1 "
result = sqlContext.sql(query)
Ans = result.map(lambda a: str(a.Ans))
print "v. Which store location had the lowest total sales?"
print Ans.collect()[0]
print ''

i. What is the average price of the products that were purchased via Mastercard?
275.067731742

ii. Which date recorded the highest total sales?
2012-03-17

iii. What is the minimum value of a product under the Computers category?
0.38

iv. How many distinct categories of products are there?
18

v. Which store location had the lowest total sales?
Plano



# Problem 3

In [58]:
purchases.take(5)

[(u'2012-07-20', u'Corpus Christi', u'CDs', 327.91, u'Cash'),
 (u'2012-03-11', u'Durham', u'Books', 115.09, u'Discover'),
 (u'2012-07-31', u'Rochester', u'Toys', 332.07, u'MasterCard'),
 (u'2012-06-18', u'Garland', u'Computers', 31.99, u'Visa'),
 (u'2012-03-27', u'Tulsa', u'CDs', 452.18, u'Discover')]

In [61]:
purchasesDF = purchases.toDF(['timestamp', 'location', 'category', 'price', 'card'])
purchasesDF.show(5)

timestamp  location       category  price  card      
2012-07-20 Corpus Christi CDs       327.91 Cash      
2012-03-11 Durham         Books     115.09 Discover  
2012-07-31 Rochester      Toys      332.07 MasterCard
2012-06-18 Garland        Computers 31.99  Visa      
2012-03-27 Tulsa          CDs       452.18 Discover  


In [74]:
from pyspark.sql.functions import *

# i. What is the average price of the products that were purchased via Mastercard?

print "i. What is the average price of the products that were purchased via Mastercard?"
purchasesDF.filter(purchasesDF.card == 'MasterCard').agg(avg(col('price'))).show()
print ''


# iii. What is the minimum value of a product under the Computers category?
print "iii. What is the minimum value of a product under the Computers category?"
purchasesDF.filter(purchasesDF.category == 'Computers').agg(min(col('price'))).show()
print ''


i. What is the average price of the products that were purchased via Mastercard?
AVG(price)        
275.06773195876286

iii. What is the minimum value of a product under the Computers category?
MIN(price)
0.38      

